In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# 1. 데이터 로드
df = pd.read_csv("EPL_with_Elo_Ratings.csv")

# 2. 파생 변수 생성
df['xg_ratio'] = df['h_xg'] / (df['h_xg'] + df['a_xg'])
df['home_advantage_score'] = df['elo_diff'] + df['xG_diff']
df = df.dropna(subset=['elo_home', 'elo_away', 'elo_diff'])

def compute_is_upset(row):
    if row['elo_diff'] > 0 and row['result'] == 1:
        return 1
    elif row['elo_diff'] < 0 and row['result'] == 2:
        return 1
    else:
        return 0
df['is_upset'] = df.apply(compute_is_upset, axis=1)
df['away_win_against_elo'] = ((df['elo_home'] > df['elo_away']) & (df['result'] == 1)).astype(int)

# 3. 학습용 피처 및 타겟 설정
features = ['elo_diff', 'xG_diff', 'xg_ratio', 'home_advantage_score', 'is_upset', 'away_win_against_elo']
X = df[features]
y = df['result']

# 4. Stratified K-Fold 정확도 측정
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(X, y), 1):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"Fold {fold} 정확도: {acc:.4f}")

# 5. 평균과 표준편차 출력
mean_acc = np.mean(accuracies)
std_acc = np.std(accuracies)
print(f"\n✅ 평균 정확도: {mean_acc:.4f}")
print(f"📉 표준편차: {std_acc:.4f}")


Fold 1 정확도: 0.8313
Fold 2 정확도: 0.8072
Fold 3 정확도: 0.8293
Fold 4 정확도: 0.8537
Fold 5 정확도: 0.8415
Fold 6 정확도: 0.8049
Fold 7 정확도: 0.7561
Fold 8 정확도: 0.8659
Fold 9 정확도: 0.8659
Fold 10 정확도: 0.7683

✅ 평균 정확도: 0.8224
📉 표준편차: 0.0362


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# 1. 데이터 불러오기
df = pd.read_csv("EPL_with_Elo_Ratings.csv")

# 2. 파생 변수 생성
df['xg_ratio'] = df['h_xg'] / (df['h_xg'] + df['a_xg'])
df['home_advantage_score'] = df['elo_diff'] + df['xG_diff']
df = df.dropna(subset=['elo_home', 'elo_away', 'elo_diff'])

def compute_is_upset(row):
    if row['elo_diff'] > 0 and row['result'] == 1:
        return 1
    elif row['elo_diff'] < 0 and row['result'] == 2:
        return 1
    else:
        return 0
df['is_upset'] = df.apply(compute_is_upset, axis=1)
df['away_win_against_elo'] = ((df['elo_home'] > df['elo_away']) & (df['result'] == 1)).astype(int)

# 3. 피처와 타깃 설정
features = ['elo_diff', 'xG_diff', 'xg_ratio', 'home_advantage_score', 'is_upset', 'away_win_against_elo']
target = 'result'

X = df[features]
y = df[target]

# 4. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 5. 모델 학습
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 6. 예측
y_pred = model.predict(X_test)

# 7. 결과 출력
print("✅ 정확도:", accuracy_score(y_test, y_pred))
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred))

# 8. 예측 vs 실제 결과 테이블
results_df = X_test.copy()
results_df['실제값'] = y_test.values
results_df['예측값'] = y_pred

# 결과 일부 미리 보기
print(results_df.head())

✅ 정확도: 0.8

📋 Classification Report:
              precision    recall  f1-score   support

           0       0.62      0.76      0.68        45
           1       0.78      0.55      0.65        38
           2       0.93      0.94      0.93        82

    accuracy                           0.80       165
   macro avg       0.77      0.75      0.75       165
weighted avg       0.81      0.80      0.80       165

      elo_diff   xG_diff  xg_ratio  home_advantage_score  is_upset  \
1160     60.15  1.347753  0.745631             61.497753         0   
279     115.26  2.387540  0.842471            117.647540         0   
107     -28.58 -0.203892  0.427166            -28.783892         0   
270     -38.45 -1.078872  0.132268            -39.528872         0   
444     138.84  0.391700  0.554696            139.231700         0   

      away_win_against_elo  실제값  예측값  
1160                     0    2    2  
279                      0    2    2  
107                      0    1    0  
270  